In [1]:
from pyspark.sql import SparkSession
import getpass

username = getpass.getuser()

spark = SparkSession. \
	builder. \
	config('spark.ui.port', '0'). \
	config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
	enableHiveSupport(). \
	master('yarn'). \
	getOrCreate()

In [2]:
spark

In [3]:
order_df = spark.read \
.format("csv") \
.option("header","true") \
.option("inferSchema","true") \
.load("/public/trendytech/orders_wh/*")

In [4]:
order_df.take(5)

[Row(order_id=1, order_date='2013-07-25 00:00:00.0', customer_id=11599, order_status='CLOSED'),
 Row(order_id=2, order_date='2013-07-25 00:00:00.0', customer_id=256, order_status='PENDING_PAYMENT'),
 Row(order_id=3, order_date='2013-07-25 00:00:00.0', customer_id=12111, order_status='COMPLETE'),
 Row(order_id=4, order_date='2013-07-25 00:00:00.0', customer_id=8827, order_status='CLOSED'),
 Row(order_id=5, order_date='2013-07-25 00:00:00.0', customer_id=11318, order_status='COMPLETE')]

In [5]:
order_df.createTempView("orders")

### 1.Top 15 customers who placed most number of orders

In [6]:
top_15cust_orders_df = order_df.groupby("customer_id") \
.count() \
.sort("count", ascending=False) \
.limit(15)

In [7]:
top_15cust_orders_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       5897|   16|
|      12431|   16|
|        569|   16|
|       6316|   16|
|      12284|   15|
|       4320|   15|
|       5624|   15|
|       5283|   15|
|        221|   15|
|       5654|   15|
|       6248|   14|
|       3708|   14|
|       1011|   14|
|       8652|   14|
|       4517|   14|
+-----------+-----+



In [8]:
result = spark.sql("SELECT customer_id, count(1) no_of_customer from orders group by customer_id order by count(1) desc limit 15")

In [9]:
result.show()

+-----------+--------------+
|customer_id|no_of_customer|
+-----------+--------------+
|       5897|            16|
|      12431|            16|
|        569|            16|
|       6316|            16|
|      12284|            15|
|       4320|            15|
|       5624|            15|
|       5283|            15|
|        221|            15|
|       5654|            15|
|       6248|            14|
|       3708|            14|
|       1011|            14|
|       8652|            14|
|       4517|            14|
+-----------+--------------+



### 2.Find the number of orders under each order status

In [10]:
no_of_orders_by_cat_df = order_df.groupby("order_status").count()

In [11]:
no_of_orders_by_cat_df

order_status,count
PENDING_PAYMENT,15030
COMPLETE,22899
ON_HOLD,3798
PAYMENT_REVIEW,729
PROCESSING,8275
CLOSED,7556
SUSPECTED_FRAUD,1558
PENDING,7610
CANCELED,1428


In [12]:
result1 = spark.sql("select order_status, count(order_id) count_of_orders from orders group by order_status")

In [13]:
result1.show()

+---------------+---------------+
|   order_status|count_of_orders|
+---------------+---------------+
|PENDING_PAYMENT|          15030|
|       COMPLETE|          22899|
|        ON_HOLD|           3798|
| PAYMENT_REVIEW|            729|
|     PROCESSING|           8275|
|         CLOSED|           7556|
|SUSPECTED_FRAUD|           1558|
|        PENDING|           7610|
|       CANCELED|           1428|
+---------------+---------------+



### 3.Number of active customers(customers who have placed atleast one order)

In [14]:
no_of_active_customers = order_df.select("customer_id").distinct().count()

In [15]:
no_of_active_customers

12405

##### Important observation, count is transformation and action as well. with group by  its an transformation as result will depends on group by column where as without group by its an action as reult will be single row

In [16]:
result2 = spark.sql("select count(distinct customer_id) from orders")

In [17]:
result2.show()

+---------------------------+
|count(DISTINCT customer_id)|
+---------------------------+
|                      12405|
+---------------------------+



### 4.Customer with most number of orders which are closed

In [18]:
cust_with_most_orders = order_df.filter("order_status = 'CLOSED'") \
.groupby("customer_id").count().sort("count", ascending = False).limit(1)

In [19]:
cust_with_most_orders.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1833|    6|
+-----------+-----+



In [21]:
result3 = spark.sql("select customer_id,count(order_id) as count from orders where order_status='CLOSED' group by customer_id order by count desc limit 1")

In [23]:
result3.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1833|    6|
+-----------+-----+

